In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import requests
from time import sleep
from pprint import pprint
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe

In [2]:
# !pip3 install gspread

In [3]:
# !pip3 install gspread-dataframe

In [4]:
# !pip3 install oauth2client

In [5]:
# お決まりの文句
# 2つのAPIを記述しないとリフレッシュトークンを3600秒毎に発行し続けなければならない
scope = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive']
#ダウンロードしたjsonファイル名をクレデンシャル変数に設定。
credentials = Credentials.from_service_account_file("makemoney-2023-540a3b015625.json", scopes=scope)
#OAuth2の資格情報を使用してGoogle APIにログイン。
gc = gspread.authorize(credentials)
#スプレッドシートIDを変数に格納する。
SPREADSHEET_KEY = '1xFbD5akY5GtJRR52x5lJkaq7zU7tTeQ9yExcYV1jAGg'
# スプレッドシート（ブック）を開く
workbook = gc.open_by_key(SPREADSHEET_KEY)

In [6]:
url = "https://suumo.jp/jj/bukken/ichiran/JJ010FJ001/?ar=030&bs=010&ta=13&firstFlg=0&urlFlg=0&jspIdFlg=1&sc=13103&kb=1&kt=9999999&km=1&mb=0&mt=9999999&ekTjCd=&ekTjNm=&tj=0&srch_navi=1&page={}"
#変数d_listに格納
d_list = []

In [7]:
# 情報取得（1～3ページ）
for i in range(1,4):
    # print('d_listの大きさ:', len(d_list))
    target_url = url.format(i)
    print(target_url)

    #取得情報の解析
    r = requests.get(target_url)
    
    sleep(1.5)
    
    soup = BeautifulSoup(r.text)
    
    #全ての物件情報取得
    contents = soup.find_all('div', class_='cassette-content js-normalLink js-cassetLink js-cassette_content')
    #各物件情報のループ取得
    for content in contents:
        #物件情報と部屋情報等
        header = content.find('h2')
        detail = content.find('div', class_='cassette-result_detail')

        title = header.find('a', class_='cassette_header-title js-cassetLinkHref js-cassette_title').text
        address = detail.find_all('p')[1].text
        access = detail.find_all('p')[3].text
        handover = detail.find_all('p')[5].text
        price = detail.find('span', class_='cassette_price-accent')
        layout = detail.find('p', class_='cassette_price-description')

        d = {
        'title':title,
        'address':address,
        'access':access,
        'handover':handover,
        'price':price.text,
        'layout':layout.text,
        }

        d_list.append(d)

https://suumo.jp/jj/bukken/ichiran/JJ010FJ001/?ar=030&bs=010&ta=13&firstFlg=0&urlFlg=0&jspIdFlg=1&sc=13103&kb=1&kt=9999999&km=1&mb=0&mt=9999999&ekTjCd=&ekTjNm=&tj=0&srch_navi=1&page=1
https://suumo.jp/jj/bukken/ichiran/JJ010FJ001/?ar=030&bs=010&ta=13&firstFlg=0&urlFlg=0&jspIdFlg=1&sc=13103&kb=1&kt=9999999&km=1&mb=0&mt=9999999&ekTjCd=&ekTjNm=&tj=0&srch_navi=1&page=2
https://suumo.jp/jj/bukken/ichiran/JJ010FJ001/?ar=030&bs=010&ta=13&firstFlg=0&urlFlg=0&jspIdFlg=1&sc=13103&kb=1&kt=9999999&km=1&mb=0&mt=9999999&ekTjCd=&ekTjNm=&tj=0&srch_navi=1&page=3


In [8]:
d_list[:2]

[{'title': 'ピアース麻布EAST',
  'address': '港区東麻布２',
  'access': '都営大江戸線/赤羽橋 徒歩4分',
  'handover': '2024年9月下旬予定',
  'price': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t価格未定',
  'layout': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1LDK～2LDK\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t/\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t31.08m2～75.62m2'},
 {'title': 'ブランズタワー芝浦',
  'address': '港区芝浦２',
  'access': 'ＪＲ山手線/田町 徒歩8分',
  'handover': '即引渡可',
  'price': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1億2850万円～1億8760万円',
  'layout': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK～3LDK\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t/\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t55.05m2～87.95m2'}]

In [9]:
df = pd.DataFrame(d_list)

In [10]:
df.price

0              \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t価格未定
1     \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1億2850万円～1...
2              \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t価格未定
3              \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t価格未定
4     \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t5920万円～1億4...
                            ...                        
68         \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t3億4480万円
69         \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t4億7800万円
70    \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1億6470万円・1...
71         \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2億7500万円
72         \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2億9500万円
Name: price, Length: 73, dtype: object

In [11]:
price_df = pd.DataFrame(df.price, columns=["price"])
price_df

,price
0,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t価格未定
1,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1億2850万円～1...
2,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t価格未定
3,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t価格未定
4,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t5920万円～1億4...
...,...
68,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t3億4480万円
69,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t4億7800万円
70,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1億6470万円・1...
71,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2億7500万円


In [12]:
price_df["upper_price"] = pd.NA
price_df["bottom_price"] = pd.NA
price_df

,price,upper_price,bottom_price
0,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t価格未定,NaN,NaN
1,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1億2850万円～1...,NaN,NaN
2,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t価格未定,NaN,NaN
3,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t価格未定,NaN,NaN
4,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t5920万円～1億4...,NaN,NaN
...,...,...,...
68,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t3億4480万円,NaN,NaN
69,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t4億7800万円,NaN,NaN
70,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1億6470万円・1...,NaN,NaN
71,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2億7500万円,NaN,NaN


In [13]:
import re

for idx, one_price in enumerate(price_df["price"]):
    # print(one_price, re.split("・|～", one_price))
    one_price = re.sub("\r|\n|\t|億|万円", "", one_price)
    one_price_list = re.split("・|～", one_price)
    one_price_list_len = len(one_price_list)
    
    if one_price_list_len == 1:
        if one_price_list[0] == "価格未定":
            price_df["price"][idx] = pd.NA
            price_df["upper_price"][idx] = pd.NA
            price_df["bottom_price"][idx] = pd.NA
        else:
            price_df["price"][idx] = one_price_list[0]
            price_df["upper_price"][idx] = re.sub("億|台／予定|台", "", one_price_list[0])
            price_df["bottom_price"][idx] = re.sub("億|台／予定|台", "", one_price_list[0])
    elif one_price_list_len == 2:
        price_df["price"][idx] = one_price_list[0] + "～" + one_price_list[1]
        price_df["upper_price"][idx] = re.sub("億|台／予定|台", "", one_price_list[0])
        price_df["bottom_price"][idx] = re.sub("億|台／予定|台", "", one_price_list[1])
    else:
        price_df["price"][idx] = "error"
        price_df["upper_price"][idx] = "error"
        price_df["bottom_price"][idx] = "error"
price_df

,price,upper_price,bottom_price
0,<NA>,<NA>,<NA>
1,12850～18760,12850,18760
2,<NA>,<NA>,<NA>
3,<NA>,<NA>,<NA>
4,5920～14790,5920,14790
...,...,...,...
68,34480,34480,34480
69,47800,47800,47800
70,16470～16880,16470,16880
71,27500,27500,27500


In [167]:
df.layout

0     \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1LDK～2LDK\r\...
1     \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK～3LDK\r\...
2     \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1LDK\r\n\t\t...
3     \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1LDK～3LDK\r\...
4     \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1DK～2LDK\r\n...
                            ...                        
68    \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK\r\n\t\t...
69    \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK\r\n\t\t...
70    \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK・3LDK\r\...
71    \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t3LDK\r\n\t\t...
72    \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t3LDK\r\n\t\t...
Name: layout, Length: 73, dtype: object

In [223]:
layout_df = pd.DataFrame(df.layout, columns=["layout"])
layout_df

,layout
0,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1LDK～2LDK\r\...
1,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK～3LDK\r\...
2,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1LDK\r\n\t\t...
3,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1LDK～3LDK\r\...
4,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1DK～2LDK\r\n...
...,...
68,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK\r\n\t\t...
69,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK\r\n\t\t...
70,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK・3LDK\r\...
71,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t3LDK\r\n\t\t...


In [224]:
layout_df["floor_plan"] = pd.NA
layout_df["area"] = pd.NA
layout_df

,layout,floor_plan,area
0,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1LDK～2LDK\r\...,NaN,NaN
1,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK～3LDK\r\...,NaN,NaN
2,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1LDK\r\n\t\t...,NaN,NaN
3,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1LDK～3LDK\r\...,NaN,NaN
4,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1DK～2LDK\r\n...,NaN,NaN
...,...,...,...
68,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK\r\n\t\t...,NaN,NaN
69,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK\r\n\t\t...,NaN,NaN
70,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK・3LDK\r\...,NaN,NaN
71,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t3LDK\r\n\t\t...,NaN,NaN


In [227]:
import re

for idx, one_layout in enumerate(layout_df["layout"]):
    # print(one_price, re.split("・|～", one_price))
    one_layout_list = re.split(" / ", one_layout)    
    # one_layout = re.sub("\r|\n|\t", "", one_layout)
    # one_layout_list = re.split(r"[ / ]", one_layout)
    # one_layout_list_len = len(one_layout_list)
    # if one_price_list_len == 1:
    layout_df["layout"][idx] = one_layout_list[0]
    layout_df["floor_plan"][idx] = re.sub("\r|\n|\t", "", one_layout_list[0])
    layout_df["area"][idx] = re.sub("\r|\n|\t", "", one_layout_list[0])

layout_df

,layout,floor_plan,area
0,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1LDK～2LDK\r\...,1LDK～2LDK/31.08m2～75.62m2,1LDK～2LDK/31.08m2～75.62m2
1,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK～3LDK\r\...,2LDK～3LDK/55.05m2～87.95m2,2LDK～3LDK/55.05m2～87.95m2
2,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1LDK\r\n\t\t...,1LDK/40.27m2～47.25m2,1LDK/40.27m2～47.25m2
3,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1LDK～3LDK\r\...,1LDK～3LDK/40.15m2～148.21m2,1LDK～3LDK/40.15m2～148.21m2
4,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1DK～2LDK\r\n...,1DK～2LDK/33.73m2～72.28m2,1DK～2LDK/33.73m2～72.28m2
...,...,...,...
68,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK\r\n\t\t...,2LDK/107.93m2（壁芯）,2LDK/107.93m2（壁芯）
69,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK\r\n\t\t...,2LDK/103.1m2（31.18坪）（壁芯）,2LDK/103.1m2（31.18坪）（壁芯）
70,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK・3LDK\r\...,2LDK・3LDK/71.02m2・74.58m2,2LDK・3LDK/71.02m2・74.58m2
71,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t3LDK\r\n\t\t...,3LDK/85.9m2（壁芯）,3LDK/85.9m2（壁芯）


In [166]:
import re

for idx, one_layout in enumerate(layout_df["layout"]):
    # print(one_price, re.split("・|～", one_price))
    one_layout = re.sub("\r|\n|\t", "", one_layout)
    one_layout_list = re.split(" / ", one_layout)
    one_layout_list_len = len(one_layout_list)
    if one_price_list_len == 1:
        layout_df["layout"][idx] = one_layout_list[0]
        layout_df["floor_plan"][idx] = re.sub(" ", "", one_layout_list[0])
        layout_df["area"][idx] = re.sub(" ", "", one_layout_list[0])

layout_df

,layout,floor_plan,area
0,1LDK～2LDK/31.08m2～75.62m2,1LDK～2LDK/31.08m2～75.62m2,1LDK～2LDK/31.08m2～75.62m2
1,2LDK～3LDK/55.05m2～87.95m2,2LDK～3LDK/55.05m2～87.95m2,2LDK～3LDK/55.05m2～87.95m2
2,1LDK/40.27m2～47.25m2,1LDK/40.27m2～47.25m2,1LDK/40.27m2～47.25m2
3,1LDK～3LDK/40.15m2～148.21m2,1LDK～3LDK/40.15m2～148.21m2,1LDK～3LDK/40.15m2～148.21m2
4,1DK～2LDK/33.73m2～72.28m2,1DK～2LDK/33.73m2～72.28m2,1DK～2LDK/33.73m2～72.28m2
...,...,...,...
68,2LDK/107.93m2（壁芯）,2LDK/107.93m2（壁芯）,2LDK/107.93m2（壁芯）
69,2LDK/103.1m2（31.18坪）（壁芯）,2LDK/103.1m2（31.18坪）（壁芯）,2LDK/103.1m2（31.18坪）（壁芯）
70,2LDK・3LDK/71.02m2・74.58m2,2LDK・3LDK/71.02m2・74.58m2,2LDK・3LDK/71.02m2・74.58m2
71,3LDK/85.9m2（壁芯）,3LDK/85.9m2（壁芯）,3LDK/85.9m2（壁芯）


In [165]:
layout_df["floor_plan"] = pd.NA
layout_df["area"] = pd.NA
layout_df

,layout,floor_plan,area
0,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1LDK～2LDK\r\...,NaN,NaN
1,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK～3LDK\r\...,NaN,NaN
2,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1LDK\r\n\t\t...,NaN,NaN
3,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1LDK～3LDK\r\...,NaN,NaN
4,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1DK～2LDK\r\n...,NaN,NaN
...,...,...,...
68,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK\r\n\t\t...,NaN,NaN
69,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK\r\n\t\t...,NaN,NaN
70,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK・3LDK\r\...,NaN,NaN
71,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t3LDK\r\n\t\t...,NaN,NaN


In [166]:
import re

for idx, one_layout in enumerate(layout_df["layout"]):
    # print(one_price, re.split("・|～", one_price))
    one_layout = re.sub("\r|\n|\t", "", one_layout)
    one_layout_list = re.split(" / ", one_layout)
    one_layout_list_len = len(one_layout_list)
    if one_price_list_len == 1:
        layout_df["layout"][idx] = one_layout_list[0]
        layout_df["floor_plan"][idx] = re.sub(" ", "", one_layout_list[0])
        layout_df["area"][idx] = re.sub(" ", "", one_layout_list[0])

layout_df

,layout,floor_plan,area
0,1LDK～2LDK/31.08m2～75.62m2,1LDK～2LDK/31.08m2～75.62m2,1LDK～2LDK/31.08m2～75.62m2
1,2LDK～3LDK/55.05m2～87.95m2,2LDK～3LDK/55.05m2～87.95m2,2LDK～3LDK/55.05m2～87.95m2
2,1LDK/40.27m2～47.25m2,1LDK/40.27m2～47.25m2,1LDK/40.27m2～47.25m2
3,1LDK～3LDK/40.15m2～148.21m2,1LDK～3LDK/40.15m2～148.21m2,1LDK～3LDK/40.15m2～148.21m2
4,1DK～2LDK/33.73m2～72.28m2,1DK～2LDK/33.73m2～72.28m2,1DK～2LDK/33.73m2～72.28m2
...,...,...,...
68,2LDK/107.93m2（壁芯）,2LDK/107.93m2（壁芯）,2LDK/107.93m2（壁芯）
69,2LDK/103.1m2（31.18坪）（壁芯）,2LDK/103.1m2（31.18坪）（壁芯）,2LDK/103.1m2（31.18坪）（壁芯）
70,2LDK・3LDK/71.02m2・74.58m2,2LDK・3LDK/71.02m2・74.58m2,2LDK・3LDK/71.02m2・74.58m2
71,3LDK/85.9m2（壁芯）,3LDK/85.9m2（壁芯）,3LDK/85.9m2（壁芯）


In [18]:
df.head()

,title,address,access,handover,price,layout
0,ピアース麻布EAST,港区東麻布２,都営大江戸線/赤羽橋 徒歩4分,2024年9月下旬予定,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t価格未定,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1LDK～2LDK\r\...
1,ブランズタワー芝浦,港区芝浦２,ＪＲ山手線/田町 徒歩8分,即引渡可,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1億2850万円～1...,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK～3LDK\r\...
2,プラウドタワー芝浦,港区芝浦４,ＪＲ山手線/田町 徒歩11分,2023年6月下旬予定,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t価格未定,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1LDK\r\n\t\t...
3,WORLD TOWER RESIDENCE,港区浜松町２,ＪＲ山手線/浜松町 徒歩2分,2025年3月下旬予定,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t価格未定,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1LDK～3LDK\r\...
4,リビオレゾン三田,港区芝５,都営三田線/三田 徒歩4分,2023年3月下旬予定,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t5920万円～1億4...,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1DK～2LDK\r\n...


In [19]:
df.shape

(73, 6)

In [20]:
df.to_csv('test.csv', index=None, encoding='utf-8-sig')

In [21]:
# # スプレッドシート（ブック）を開く
# workbook = gc.open_by_key(SPREADSHEET_KEY)
# # シートの一覧を取得する。（リスト形式）
# worksheets = workbook.worksheets()
# print(worksheets)
# # シートを開く
# worksheet = workbook.worksheet('シート1')
# # セルA1に”test value”という文字列を代入する。
# worksheet.update_cell(1, 1, 'test value')

In [22]:
# スプレッドシート（ブック）を開く
workbook = gc.open_by_key(SPREADSHEET_KEY)

# シートの一覧を取得する。（リスト形式）
worksheets = workbook.worksheets()
print(worksheets)

# シートを開く
worksheet = workbook.worksheet('シート1')

# 作ったDataFrameを貼り付ける。
set_with_dataframe(worksheet, df, include_index=None)

[<Worksheet 'シート1' id:0>]


In [23]:
# worksheet.clear()

In [24]:
df.isnull().any()

title       False
address     False
access      False
handover    False
price       False
layout      False
dtype: bool

In [25]:
df.isnull().sum()

title       0
address     0
access      0
handover    0
price       0
layout      0
dtype: int64

In [ ]:
df